# Imports

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
import ipyplot
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load Data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 4us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 110s 4us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 18s 4us/step


# EDA

In [4]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [8]:
def display_images(images, labels, with_tabs=False):
    class_names = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    labels = [class_names[label] for label in labels]

    if with_tabs:
        ipyplot.plot_class_tabs(images, labels, max_imgs_per_tab=10, img_width=150)
    else:
        ipyplot.plot_images(images, labels, max_images=20)

display_images(x_train[:20], y_train[:20])

In [9]:
display_images(x_train[:30], y_train[:30], with_tabs=True)

In [10]:
def save_images(images, labels, save_dir="../../datasets/images/fashion_mnist"):
    os.makedirs(save_dir, exist_ok=True)
    class_names = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    label_names = [class_names[label] for label in labels]

    for i, image in enumerate(images):
        # Convert NumPy array to PIL Image
        pil_image = Image.fromarray(image)
        
        # Generate a filename based on class name and index
        filename = f"{label_names[i]}_{i + 1}.png"
        
        # Save the image in the specified folder
        pil_image.save(os.path.join(save_dir, filename))
    
    print(f"Images saved in the folder: {save_dir}")

save_images(x_train[:10000], y_train[:10000])

Images saved in the folder: ../../datasets/images/fashion_mnist


In [13]:
print(f"Min pixel value: {x_train.min()}")
print(f"Max pixel value: {x_train.max()}") 

Min pixel value: 0
Max pixel value: 255


# Preprocessing, training and evaluation

In [16]:
# Normalization and reshaping to add channel dimension
x_train = x_train.reshape((-1,28,28,1)).astype("float32") / 255.0
x_test = x_test.reshape((-1,28,28,1)).astype("float32") / 255.0

In [17]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)


In [23]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
             )

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
hist = model.fit(x_train, y_train, epochs=15, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 27s 26ms/step - accuracy: 0.4436 - loss: 1.5309 - val_accuracy: 0.7236 - val_loss: 0.7706
Epoch 2/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 39s 24ms/step - accuracy: 0.7305 - loss: 0.7177 - val_accuracy: 0.7486 - val_loss: 0.6703
Epoch 3/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.7570 - loss: 0.6424 - val_accuracy: 0.7685 - val_loss: 0.6190
Epoch 4/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.7780 - loss: 0.5876 - val_accuracy: 0.7882 - val_loss: 0.5735
Epoch 5/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.7947 - loss: 0.5369 - val_accuracy: 0.8055 - val_loss: 0.5353
Epoch 6/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 24s 25ms/step - accuracy: 0.8113 - loss: 0.5042 - val_accuracy: 0.8045 - val_loss: 0.5317
Epoch 7/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 25ms/step - accuracy: 0.8174 - loss: 0.4884 - val_accuracy: 0.8197 - val_loss: 0.4951
Epoch 8/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 25s 26ms/step - accuracy: 0.8265 - loss: 0.4727 - 

In [25]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_acc}")

313/313 - 2s - 6ms/step - accuracy: 0.8463 - loss: 0.4259
Test loss: 0.42594051361083984
Test accuracy: 0.8463000059127808


In [ ]:
# Predict on the test set
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [45]:
print("Training Accuracy:", hist.history['accuracy'][-1])
print("Validation Accuracy:", hist.history['val_accuracy'][-1])

Training Accuracy: 0.8560500144958496
Validation Accuracy: 0.8463000059127808


In [26]:
model_dir="../../models"
os.makedirs(model_dir, exist_ok=True)
model.save(f"{model_dir}/fashion_mnist_cnn_model.keras")

# Gradio UI

In [29]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('../../models/fashion_mnist_cnn_model.keras')

# Class names corresponding to Fashion_mnist labels
class_names = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']

# Preprocess the uploaded image
def preprocess_image(img):
    img = img.convert('L') # to grayscale
    img = img.resize((28,28)) # (28,28)
    img = np.array(img)
    
    # Normalize the image to the range [0, 1]
    img = img / 255.0

    # Add channel dimension
    img = np.expand_dims(img, axis=-1) # (28, 28, 1)
    
    # Expand dimensions to add batch size (1 image)
    img = np.expand_dims(img, axis=0).astype('float32') # (1, 28, 28, 1)
    
    return img

# Prediction function
def predict_image(img):
    if img is None:
        return None
    img = preprocess_image(img)
    predictions = model.predict(img)
    print(predictions)
    predicted_class = np.argmax(predictions)
    return class_names[predicted_class]

demo = gr.Interface(fn=predict_image, 
                    inputs=gr.Image(type="pil", label="Upload an image"), 
                    outputs=gr.Label(num_top_classes=1, label="Predicted Class"),
                    title = "Fashion MNIST Classification",
                    description = "Upload a grayscale or RGB img and the model will classify iy as one of the Fashion MNIST categories",
                    live=True)

demo.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
